# Example 3+4 Models in Docker

该Example展示了如何使用一个model提供*实时预测*服务，包括两种部署方式

* 使用Flask+Tensorflow部署
* 使用Tensorflow Serving部署

本Notebook当中比较了两种部署方式的性能，可以看到，
同样的tensorflow模型，使用Tensorflow Serving的部署方式具有显著的性能优势

## Step 1 训练一个模型

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.3.1'

In [2]:
# load titanic data
# https://www.kaggle.com/c/titanic/data
train_data = pd.read_csv('./titanic_train.csv',index_col=0)
train_data['Embarked'] = train_data['Embarked'].astype(str)
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# train a keras model
# with tensorflow features

# define features
features = [
    tf.feature_column.numeric_column('Age', default_value=train_data.Age.mean()),
    tf.feature_column.numeric_column('Fare', default_value=train_data.Fare.mean()),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list('Embarked', train_data.Embarked.unique()),
    ),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list('Sex', train_data.Sex.unique()),
    ),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_identity('Pclass', num_buckets=train_data.Pclass.max()),
    )
]

feature_layer = keras.layers.DenseFeatures(features)

model = keras.Sequential([
    feature_layer,
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='Adam')

X = train_data[[
    'Age','Fare','Embarked','Sex','Pclass'
]].to_dict('series')

for key in X:
    if X[key].dtype == np.float32 or X[key].dtype == np.float64:
        X[key] = X[key].fillna(X[key].mean()).values
    else:
        X[key] = X[key].values
    
y = train_data.Survived.values

model.fit(X,y)
model.summary()

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
28/28 [==============================] - 0s 1ms/step - loss: 2.4243
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  384       
_________________________________________________________________
dense_1 (Dense)              multiple                  33        
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [4]:
keras.models.save_model(model, './tf_serving/model',save_format='tf')
keras.models.save_model(model, './flask/model',save_format='tf')

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
INFO:tensorflow:Assets written to: ./tf_serving/model/assets
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functi

## Step 2 Deploy Tensorflow Serving Docker

In [5]:
!docker build --tag tf_serving ./tf_serving/

[+] Building 0.0s (0/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.1s (6/6)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 36B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/tensorflow/serving:latest       0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 239.99kB                                      0.0s
 => CACHED [1/2] FROM docker.io/tensorflow/serving                         0.0s
 => [2/2] COPY ./model /models/model/1                                     0.0s
[+] Building 0.1s (7/7) FINISHED      

In [6]:
!docker run --name exp3_tf_serving -p 8081:8501 -d tf_serving

732380b75a9a5be5dabe028fa934a82cf41f78b38de3cfbc79cf53430a12e1bd


## Step 3 Deploy Flask Docker Container

In [7]:
!docker build --tag flask ./flask/

[+] Building 0.0s (2/2)                                                         
 => [internal] load .dockerignore                                          0.0s
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.2s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 129B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.7-slim         0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition 

[+] Building 2.4s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 129B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.7-slim         2.4s
[+] Building 2.6s (2/3)                                                         
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 129B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.7-slim         2.5s
[+] Building 2.7s (2/3)               

In [8]:
!docker run --name exp3_flask -p 8080:8080 -d flask

4f91390a3bc52036a5045c71511f407876c3dcad162b0dee9e6c8013d109e865


## Step 4 Compare Model Performance

In [9]:
import requests
from tqdm.auto import tqdm
from time import time

In [10]:
test_data = pd.read_csv('./titanic_test.csv',index_col=0)
test_data = test_data[[
    'Age','Fare','Embarked','Sex','Pclass'
]]

In [11]:
test_data.Sex.value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [12]:
train_data.Sex.unique()

array(['male', 'female'], dtype=object)

In [13]:
# tensorflow serving
start = time()
for _, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    item = row.to_dict()
    for key in item:
        if 'float' in str(type(item[key])):
            item[key] = float(item[key])
        if 'int' in str(type(item[key])):
            item[key] = int(item[key])
        
        item[key] = [item[key]]
            
    res = requests.post('http://localhost:8081/v1/models/model:predict',json = {'instances': [item]})
    res.raise_for_status()
    
end = time()
'Used', end-start, 'seconds'

('Used', 4.414155960083008, 'seconds')

In [14]:
# flask with model
start = time()
for _, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    item = row.to_dict()
    for key in item:
        if 'float' in str(type(item[key])):
            item[key] = float(item[key])
        if 'int' in str(type(item[key])):
            item[key] = int(item[key])

    res = requests.post('http://localhost:8080/model',json = item)
    res.raise_for_status()
end = time()
'Used', end-start, 'seconds'

('Used', 28.770965099334717, 'seconds')

In [15]:
# clean environment

In [16]:
!docker stop exp3_flask

exp3_flask


In [17]:
!docker rm exp3_flask

exp3_flask


In [18]:
!docker stop exp3_tf_serving

exp3_tf_serving


In [19]:
!docker rm exp3_tf_serving

exp3_tf_serving
